## LightGBM算法梳理

### 1 LightGBM

In [ ]:
提升树是利用加法模型与前向分步算法实现学习的优化过程，
它有一些高效实现，如XGBoost, pGBRT，GBDT等。
其中GBDT采用负梯度作为划分的指标（信息增益），XGBoost则利用到二阶导数。
它们共同的不足是计算信息增益需要扫描所有样本，从而找到最优划分点。
在面对大量数据或者特征维度很高时，它们的效率和扩展性很难使人满意。


### 2 LightGBM的起源

In [ ]:
微软开源的LightGBM（基于GBDT的）则很好的解决这些问题，
LightGBM是个快速的，分布式的，高性能的基于决策树算法的梯度提升框架。

有以下优势：
    更快的训练效率;
    低内存使用;
    更高的准确率;
    支持并行化学习;
    可处理大规模数据;

In [ ]:
XGBoost和LightGBM之间更加细致的性能对比如下所示

In [1]:
%%html
<img src='4.png'>

In [ ]:
xgboost优缺点:

优点:XGB利用了二阶梯度来对节点进行划分，相对来说，精度更加高。XGB利用局部近似算法对分裂节点的贪心算法优化，取适当的eps时，可以保持算法的性能且提高算法的运算速度。在损失函数中加入了L1/L2项，控制模型的复杂度，提高模型的鲁棒性。提供并行计算能力，主要是在树节点求不同的候选的分裂点的Gain Infomation（分裂后，损失函数的差值）,Tree Shrinkage，column subsampling等不同的处理细节。

缺点:每轮迭代时，都需要多次遍历整个训练数据。如果把整个训练数据装进内存则会限制训练数据的大小；如果不装进内存，反复地读写训练数据又会消耗非常大的时间。

预排序方法（pre-sorted）：首先，空间消耗大。这样的算法需要保存数据的特征值，还保存了特征排序的结果（例如排序后的索引，为了后续快速的计算分割点），这里需要消耗训练数据两倍的内存。其次时间上也有较大的开销，在遍历每一个分割点的时候，都需要进行分裂增益的计算，消耗的代价大。

对cache优化不友好。在预排序后，特征对梯度的访问是一种随机访问，并且不同的特征访问的顺序不一样，无法对cache进行优化。同时，在每一层长树的时候，需要随机访问一个行索引到叶子索引的数组，并且不同特征访问的顺序也不一样，也会造成较大的cache miss。

LightGBM改进:

针对这些缺点LightGBM进行了相应的改进。LightGBM基于histogram算法代替pre-sorted所构建的数据结构，利用histogram做差，提高了cache命中率（主要是因为使用了leaf-wise）。

在机器学习当中，面对大数据量时候都会使用采样的方式（根据样本权值）来提高训练速度。又或者在训练的时候赋予样本权值来处理某一类样本（如Adaboost）。LightGBM利用了GOSS来做采样算法。

 由于histogram算法对稀疏数据的处理时间复杂度没有pre-sorted好,因为histogram并不管特征值是否为0。因此采用了EFB来预处理稀疏数据。


### 3 Histogram VS pre-sorted

In [ ]:
Histogram算法:

直方图算法的基本思想：先把连续的浮点特征值离散化成k个整数，同时构造一个宽度为k的直方图。遍历数据时，根据离散化后的值作为索引在直方图中累积统计量，当遍历一次数据后，直方图累积了需要的统计量，然后根据直方图的离散值，遍历寻找最优的分割点。

相对于pre-sorted算法，它的内存空间需要相对小很多。因为pre-sorted算法需要保存每一个特征的排序结构，所以其需要的内存大小是2 * #data * #feature * 4Bytes,而histogram只需保存离散值bin value而且不需要原始的feature value，所以占用的内存大小为：#data * # feature * 1Byte，因为离散值bin value使用uint8_t已经足够了。另外对于求子节点相应的feature histogram时，只需构造一个子节点的feature histogram，另外一个子节点的feature histogram可以用父节点的histogram减去刚构造出来的子节点的histogram便可，时间复杂度就压缩到O(k)，k为histogram的桶数。这是一个很巧妙的做差法。


### 4 leaf-wise VS level-wise

In [ ]:
LightGBM使用带深度限制的Leaf-wise的叶子生长策略.

Level-wise遍历一次数据可以同时分裂同一层的叶子，容易进行多线程优化，也好控制模型复杂度，不容易过拟合。但实际上Level-wise是一种低效算法，因为它不加区分的对待同一层的叶子，带来了很多没必要的开销，因为实际上很多叶子的分裂增益较低，没必要进行搜索和分裂。

Leaf-wise则是一种更为高效的策略：每次从当前所有叶子中，找到分裂增益最大的一个叶子，然后分裂，如此循环。因此同Level-wise相比，在分裂次数相同的情况下，Leaf-wise可以降低更多的误差，得到更好的精度。

Leaf-wise的缺点：可能会长出比较深的决策树，产生过拟合。因此LightGBM在Leaf-wise之上增加了一个最大深度限制，在保证高效率的同时防止过拟合。


### 5 特征并行和数据并行

In [ ]:
特征并行的主要思想是在不同机器在不同的特征集合上分别寻找最优的分割点，
然后在机器间同步最优的分割点。

数据并行则是让不同的机器先在本地构造直方图，
然后进行全局的合并，
最后在合并的直方图上面寻找最优分割点。

### 6 顺序访问梯度

### 7 支持类别特征

In [ ]:
为了解决one-hot编码处理类别特征的不足。LGBM采用了Many vs many的切分方式，实现了类别特征的最优切分。用Lightgbm可以直接输入类别特征，并产生如图1右边的效果。在1个k维的类别特征中寻找最优切分，朴素的枚举算法的复杂度是，而LGBM采用了如参考文献【1】的方法实现了的算法。

算法流程如图2所示：在枚举分割点之前，先把直方图按每个类别的均值进行排序；然后按照均值的结果依次枚举最优分割点。从图2可以看到，Sum(y)/Count(y)为类别的均值。当然，这个方法很容易过拟合，所以在LGBM中加入了很多对这个方法的约束和正则化。图3是一个简单的对比实验，可以看到该最优方法在AUC上提升了1.5个点，并且时间只多了20%。


In [2]:
%%html
<img src='5.jpg'>

In [ ]:
下面具体来讲下在代码中如何求解类别特征的最优切分的流程：

（feature_histogram.hpp文件中FindBestThresholdCategorical函数）

A. 离散特征建立直方图的过程：

统计该特征下每一种离散值出现的次数，并从高到低排序，并过滤掉出现次数较少的特征值, 然后为每一个特征值，建立一个bin容器, 对于在bin容器内出现次数较少的特征值直接过滤掉，不建立bin容器。

B. 计算分裂阈值的过程：

B.1
先看该特征下划分出的bin容器的个数，如果bin容器的数量小于4，直接使用one vs other方式, 逐个扫描每一个bin容器，找出最佳分裂点;

B.2
对于bin容器较多的情况, 先进行过滤，只让子集合较大的bin容器参加划分阈值计算, 对每一个符合条件的bin容器进行公式计算(公式如下: 该bin容器下所有样本的一阶梯度之和 / 该bin容器下所有样本的二阶梯度之和 + 正则项(参数cat_smooth)，这里为什么不是label的均值呢？其实上例中只是为了便于理解，只针对了学习一棵树且是回归问题的情况， 这时候一阶导数是Y, 二阶导数是1)，得到一个值，根据该值对bin容器从小到大进行排序，然后分从左到右、从右到左进行搜索，得到最优分裂阈值。但是有一点，没有搜索所有的bin容器，而是设定了一个搜索bin容器数量的上限值，程序中设定是32，即参数max_num_cat。
LightGBM中对离散特征实行的是many vs many 策略，这32个bin中最优划分的阈值的左边或者右边所有的bin容器就是一个many集合，而其他的bin容器就是另一个many集合。

B.3
对于连续特征，划分阈值只有一个，对于离散值可能会有多个划分阈值，每一个划分阈值对应着一个bin容器编号，当使用离散特征进行分裂时，只要数据样本对应的bin容器编号在这些阈值对应的bin集合之中，这条数据就加入分裂后的左子树，否则加入分裂后的右子树。



### 8 应用场景

### 9 sklearn参数

In [ ]:
LightGBM 可以使用列表或字典来设置参数：

param = {'num_leaves':31, 'num_trees':100, 'objective':'binary'}
 
param['metric'] = 'auc'
 
param['metric'] = ['auc', 'binary_logloss']
训练只需传入param即可


训练模型时, 需要参数列表和数据集:

num_round = 10
bst = lgb.train(param, train_data, num_round, valid_sets=[test_data])
param：各项参数，是一个字典

train_data：训练数据集

num_round：boosting迭代计算次数

valid_sets：验证集设置

test_data：验证数据集


### 10 CatBoost(了解)